---
title: "Predicting Chronic Kidney Disease"
subtitle: "with Machine Learning & Visual Analytics"
author: "Shreemithra Naveen and Sai Laasya Gorantla"
date: "17 April 2025"
title-slide-attributes:
      data-background-image: images/background_image1.png
      data-background-size: cover
      data-background-opacity: "0.6"
theme: serif
format:
  revealjs:    
    slide-number: true
    transition: slide
    background-transition: fade
jupyter: python3
editor: visual
execute:
  warning: false
  message: false
  echo: false
---

In [ ]:
#| label: load-packages
#| include: false

#Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [ ]:
#| label: load-data-and-preprocessing
#| include: false
disease = pd.read_csv('kidney_disease.csv')

#Dropping off the column 'id'
disease = disease.drop('id', axis = 1)

#Renaming the columns for easy understanding the column and dataset clearly
disease.columns = ['age', 'blood_pressure', 'specific_gravity', 'albumin', 'sugar', 'red_blood_cells', 'pus_cell',
'pus_cell_clumps', 'bacteria', 'blood_glucose_random', 'blood_urea', 'serum_creatinine', 'sodium','potassium', 'haemoglobin', 'packed_cell_volume', 'white_blood_cell_count', 'red_blood_cell_count',
'hypertension', 'diabetes_mellitus', 'coronary_artery_disease', 'appetite', 'peda_edema','aanemia', 'class']

#Converting the column, the errors is applied for not considering the NA values
disease[['packed_cell_volume', 'white_blood_cell_count', 'red_blood_cell_count']] = disease[['packed_cell_volume', 'white_blood_cell_count', 'red_blood_cell_count']].apply(pd.to_numeric, errors = 'coerce')

#As the 6-7 columns has just 1,2 or 4 missing values
disease.dropna(subset=['hypertension', 'diabetes_mellitus','coronary_artery_disease','appetite', 'peda_edema','aanemia', 'bacteria', 'pus_cell_clumps'], inplace = True)

#disease.isna().sum()

categorical_null = ['red_blood_cells', 'pus_cell']

values = ['normal', 'abnormal']
for feature in categorical_null:
    print(disease[feature].unique(), disease[feature].isna().sum())
    disease[feature].fillna(random.choice(values), inplace=True)

categorical = disease.select_dtypes(include = 'object')
categorical.columns 

## Replacing the incorrect values - cleaning

# Clean 'diabetes_mellitus' column
disease['diabetes_mellitus']= disease['diabetes_mellitus'].replace(to_replace={'\tno': 'no', '\tyes': 'yes', ' yes': 'yes'})
disease['coronary_artery_disease'] = disease['coronary_artery_disease'].replace(to_replace='\tno',value='no')
disease['class'] = disease['class'].replace(to_replace={'ckd\t': 'ckd'})

#Converting the class to the numerical column and also assigning binary values
disease['class'] = disease['class'].map({'ckd': 0, 'notckd': 1})
disease['class'] = pd.to_numeric(disease['class'], errors='coerce')

#plt.figure(figsize=(15,15))
#for i,col_name in enumerate(categorical.columns, start = 1):
#    plt.subplot(5,5,i)
#    sns.countplot(x = disease[col_name])
#plt.tight_layout()
#plt.show()

#Fetching the numerical columns
numerical_columns = disease.select_dtypes(exclude = 'object')

#plt.figure(figsize=(15,15))
#sns.pairplot(numerical_columns)
#plt.show()

## Project Overview {.center}

- Over **850 million people worldwide** have kidney diseases — more than diabetes and cancer combined.
- **1 in 10 people globally** are affected by CKD.
- CKD is the **8th leading cause of death** worldwide (WHO, 2020).

<span style="font-size: 0.65em; color: #555;">
Source: World Health Organization (WHO), 2020 – <a href="https://www.who.int/news-room/fact-sheets/detail/kidney-disease">Global Facts on CKD</a>
</span>


# Silent & Dangerous

------------------------------------------------------------------------

<!-- Upper half text -->
<div style="text-align: center; font-size: 0.7em; margin-bottom: 2vh;">
  <p>CKD often has no symptoms in early stages — making it a “silent disease".By the time symptoms appear, <strong>irreversible damage</strong> may have occurred.</p>
</div>

<!-- lower half text -->

:::{.columns}

::: {.column width="50%"}


In [ ]:
#| label: piechart-ckdvsnoncks

labels = ['Unaware of CKD', 'Aware of CKD']
sizes = [90, 10]
colors = ['#f4a261', '#2a9d8f']
fig, ax = plt.subplots(figsize=(3.9, 3.9), facecolor='none')
ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax.set_facecolor('none')   
plt.axis('equal')          
plt.tight_layout()
plt.show()

<p style="text-align: center; font-size: 0.50em; color: #444;">
<b>U.S. Perspective:</b><br>
9 out of 10 people with CKD are unaware they have it.<br>
<em>— American Kidney Fund</em>
</p>

:::

::: {.column width="50%"}


In [ ]:
#| label: piechart-terminalstage

labels = ['Terminal Stage', 'Early Stage']
sizes = [72.5, 27.5]
colors = ['#ba2f2f', '#87ceeb']

fig, ax = plt.subplots(figsize=(4, 4), facecolor='none')
ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax.set_facecolor('none')
plt.axis('equal')
plt.tight_layout()
plt.show()

<p style="text-align: center; font-size: 0.50em; color: #444;">
<b>Global Perspective:</b><br>
72.5% of patients were diagnosed at the terminal stage,  
and 75% had never heard of CKD before hospitalization.<br>
<em>— Brazzaville Hospital Study</em>
</p>

:::

:::

<div style="text-align: center; font-size: 0.6em; color: #666; margin-top: 1em;">
  Source: 
  <a href="https://www.kidneyfund.org" target="_blank">AKF</a> · 
  <a href="https://indexmedicus.afro.who.int/iah/fulltext/Chronic%20Renal%20Failure..." target="_blank">Brazzaville Study</a>
</div>


#  <span style="font-size: 0.6em;">Why Early Detection Matters?</span>

<p style="text-align: center; font-size: 0.70em; color: #444;">
That’s why we decided to build a <b>machine learning model</b> to assist in <b>early detection of CKD</b>, even during routine checkups — helping patients become aware of their symptoms and risks <b>before it’s too late</b><br>
</p>

<p style="text-align: center; font-size: 0.60em; color: #444;">
<b>Early detection enables:</b><br>
  - Slowing or halting disease progression<br>
  - Improved quality of life<br>
  - Reduced need for dialysis or transplant<br>
</p>

---

##  <span style="font-size: 0.8em;">Dataset Overview</span>


<p style="text-align: left; font-size: 0.60em; color: #444;">
    - Dataset: Chronic Kidney Disease dataset from [Kaggle](https://www.kaggle.com/datasets/mansoordaku/ckdisease)<br>
    - 400 patient records with 26 clinical features<br>
    - Target variable: `class` → CKD or not CKD<br>
</p>

  <p style="text-align: left; font-size: 0.90em; color: #444; margin: 0;">
    • Includes essential CKD indicators: albumin, creatinine, hemoglobin, BP<br>
    • One of the few public datasets with complete labels + clinical features<br>
    • Ideal for building a clear, interpretable ML model
  </p>



---

## <span style="font-size: 0.8em;">Categorical Features in the Dataset</span>

<p style="font-size: 0.70em; text-align: center; color: #444;">
This chart reveals how categorical variables like anemia, appetite, and diabetes are distributed across the patient records — helping us understand the clinical profiles common in CKD cases.
</p>


In [ ]:
#| label: Distribution-of-Categorical-Features

import warnings
warnings.filterwarnings("ignore")

categorical = categorical.drop(columns='class')

sns.set_style("whitegrid")
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['figure.facecolor'] = 'none'
plt.figure(figsize=(20, 18))

for i, col_name in enumerate(categorical.columns, start=1):
    plt.subplot(5, 5, i)
    sns.countplot(x=disease[col_name], hue=disease[col_name], palette="Set2", legend=False)
    plt.title(col_name.replace("_", " ").capitalize(), fontsize= 16, fontweight='bold')
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel("")
    plt.ylabel("Count", fontsize=12)

plt.suptitle("Distribution of Categorical Features in CKD Dataset", fontsize=18, fontweight='bold', y=1.02)
plt.subplots_adjust(hspace=0.7)
plt.tight_layout()
plt.show()

---


In [ ]:
#| label: Distribution-of-numerical-Features

numerical = disease.select_dtypes(include=['float64', 'int64'])
numerical = numerical.drop(columns=['class','specific_gravity', 'packed_cell_volume'])

disease['class_label'] = disease['class'].map({1: 'CKD', 0: 'Non-CKD'})

custom_palette = {'CKD': '#e74c3c', 'Non-CKD': '#2ecc71'}

cols = 6
rows = (len(numerical.columns) + cols - 1) // cols

plt.figure(figsize=(30, rows * 5))
sns.set_style("whitegrid")
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['figure.facecolor'] = 'none'

# Define custom x-limits for specific columns
xlimits = {
    'serum_creatinine': (0, 25),
    'sodium': (80, 180),
    'potassium': (0, 15),
}

for i, col in enumerate(numerical.columns, 1):
    plt.subplot(rows, cols, i)
    sns.histplot(data=disease, x=col, kde=True, hue="class_label", palette=custom_palette, legend=False)
    plt.title(col.replace('_', ' ').capitalize(), fontsize=16)
    plt.ylabel("Count", fontsize=14)                              # Add y-label
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    
    # Apply custom x-axis range if defined
    if col in xlimits:
        plt.xlim(xlimits[col])

# Add a single legend
import matplotlib.patches as mpatches
ckd_patch = mpatches.Patch(color=custom_palette['CKD'], label='CKD')
nonckd_patch = mpatches.Patch(color=custom_palette['Non-CKD'], label='Non-CKD')
plt.figlegend(handles=[ckd_patch, nonckd_patch], loc='upper left', ncol=2, fontsize=12, title="CKD Status")

# Titles and layout
plt.suptitle("Distribution of Key Numerical Features by CKD Class", fontsize=20, fontweight='bold', y=0.98)
plt.tight_layout()
plt.subplots_adjust(bottom=0.1, hspace=0.5)

plt.savefig("numerical_distribution.png")
plt.close()

## <span style="font-size: 0.8em;">Correlation Heatmap of Clinical Variables</span>


:::{.columns}


::: {.column width="55%"}

In [ ]:
#| label: heat-map
  
plt.figure(figsize=(9, 9))
sns.heatmap(numerical_columns.corr(), annot=True, cmap='Set2')
plt.title("Correlation Among Key Clinical Features", fontsize=14)
plt.tight_layout()
plt.show()

:::

::: {.column width="45%"}
<br>
<div style="display: flex; flex-direction: column; justify-content: center; height: 100%;">

<p style="font-size: 0.5em; text-align: left; color: #444;">
This heatmap shows how different clinical variables are correlated with each other.
</p> 

<p style="font-size: 0.5em; text-align: left;">
<b>1. Hemoglobin, Packed Cell Volume, and Red Blood Cell Count</b> show strong positive correlations — reflecting their interconnected role in oxygen transport and anemia, common in CKD.
</p>

<p style="font-size: 0.5em; text-align: left;">
<b>2. Serum Creatinine</b> is negatively correlated with <b>Sodium</b> and <b>Albumin</b>, indicating impaired kidney function affects electrolyte balance and protein retention.
</p>

</div>

:::

:::
---


## <span style="font-size: 0.8em;">Exploring Feature Relationships in CKD Patients </span>

<p style="font-size: 0.50em; text-align: center; color: #444;">
These plots highlight clinical feature pairs that are strongly correlated and vary by CKD status.The separation of CKD vs non-CKD patients in these pairs reveals underlying clinical patterns that can support early detection and diagnosis.
</p>


In [ ]:
#| label: Correlated-Feature-Pairs
#| fig-align: center

pairs = [
    ('haemoglobin', 'packed_cell_volume'),
    ('packed_cell_volume', 'red_blood_cell_count'),
    ('haemoglobin', 'red_blood_cell_count'),
    ('sugar', 'blood_glucose_random'),
    ('blood_urea', 'red_blood_cell_count'),
    ('blood_urea', 'haemoglobin'),
    ('albumin', 'haemoglobin'),
    ('serum_creatinine', 'sodium')
]

# Map class values to readable labels
disease['class_label'] = disease['class'].map({0: 'CKD', 1: 'Non-CKD'})

sns.set_theme(style="whitegrid", palette="viridis")
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['figure.facecolor'] = 'none'

n = len(pairs)
rows = (n + 1) // 2

# Create figure and axes
fig, axes = plt.subplots(rows, 4, figsize=(20, rows * 4))
axes = axes.flatten()

# Placeholder for handles and labels
handles, labels = None, None

for i, (x, y) in enumerate(pairs):
    ax = axes[i]
    if x in disease.columns and y in disease.columns:
        # Plot with legend only once (first valid pair)
        scatter = sns.scatterplot(data=disease, x=x, y=y, hue='class_label',
                                  palette='Set1', alpha=0.8, s=60, ax=ax, legend=(handles is None))
        sns.regplot(data=disease, x=x, y=y, scatter=False,
                    color='black', line_kws={"linewidth": 2, "linestyle": "--"}, ax=ax)

        # Save handles for legend once
        if handles is None:
            handles, labels = scatter.get_legend_handles_labels()

        # Title and labels
        corr_val = disease[[x, y]].dropna().corr().iloc[0, 1]
        ax.set_title(f"{x.replace('_', ' ').capitalize()} vs {y.replace('_', ' ').capitalize()}\n(r = {corr_val:.2f})", fontsize=16)
        ax.set_xlabel(x.replace('_', ' ').capitalize(), fontsize=14)
        ax.set_ylabel(y.replace('_', ' ').capitalize(), fontsize=14)

# Remove any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Add shared legend if we captured one
if handles and labels:
    fig.legend(handles, labels, loc='upper right', ncol=2, fontsize=12, title="CKD Status")

# Super title and spacing
plt.suptitle('CKD Dataset – Strongly Correlated Feature Pairs with Class Highlighting', fontsize=20, y=1.03)
plt.tight_layout()
plt.subplots_adjust(top=0.92, bottom=0.12)
plt.show()

## <span style="font-size: 0.8em;">Building a Machine Learning Model for CKD Prediction</span>

<p style="font-size: 0.6em; text-align: left;"> Based on our exploratory data analysis, we selected a focused set of features that showed strong correlation with CKD and had clinical relevance in early diagnosis. </p> 

<p style="font-size: 0.6em; text-align: left;"> We define our modeling variables as: </p>


In [ ]:
#| label: X-Y-display
#| echo: true
#| eval: false

X = disease[['red_blood_cell_count', 'packed_cell_volume', 'haemoglobin', 'albumin', 'specific_gravity']]
y = disease['class']

<p style="font-size: 0.6em; text-align: left;"> The objective is to build a simple, interpretable machine learning model that can assist in CKD detection using just these five features. </p> 

<div style="background-color: #e6f4ea; color: #1b5e20; padding: 10px 15px; font-size: 0.6em; border-left: 5px solid #2e7d32; border-radius: 8px;">
✅ Clinically meaningful features<br>
✅ Minimal but strong predictors<br>
✅ Useful in real-world check-ups<br>
✅ Balanced for interpretability
</div>


In [ ]:
#| label: Model-Building
#| eval: false

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


# Features selected based on exploratory analysis
X = disease[['red_blood_cell_count', 'packed_cell_volume', 'haemoglobin', 'albumin', 'specific_gravity']]
y = disease['class']

# Handle missing values (using mean imputation for simplicity)
X = X.fillna(X.mean())

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#-------SVM---------

svm_rbf = SVC(kernel='rbf', random_state=42)
svm_rbf.fit(X_train, y_train)
svm_preds = svm_rbf.predict(X_test)

print("SVM (RBF) Accuracy:", accuracy_score(y_test, svm_preds))
print(confusion_matrix(y_test, svm_preds))
print(classification_report(y_test, svm_preds))


# --- KNN Model ---
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

# Evaluation
knn_preds = knn.predict(X_test)
print(f"KNN Accuracy: {accuracy_score(y_test, knn_preds):.2f}")
print("KNN Confusion Matrix:\n", confusion_matrix(y_test, knn_preds))
print("KNN Classification Report:\n", classification_report(y_test, knn_preds))


# --- Decision Tree Classifier ---
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

# Evaluation
dtc_preds = dtc.predict(X_test)
print(f"\nDecision Tree Accuracy: {accuracy_score(y_test, dtc_preds):.2f}")
print("Decision Tree Confusion Matrix:\n", confusion_matrix(y_test, dtc_preds))
print("Decision Tree Classification Report:\n", classification_report(y_test, dtc_preds))

---

## <span style="font-size: 0.8em;">Modeling Decisions: What We Tried & Why?</span>

 <p style="font-size: 0.6em; text-align: left; color: #444;">
We tested three key machine learning models to understand their strengths in detecting CKD using core clinical features:
</p>

<ul style="font-size: 0.6em; color: #444;">
  <li><b>K-Nearest Neighbors (KNN):</b> Performed well and was simple to implement, but slightly sensitive to local data variations.</li>
  <li><b>Decision Tree:</b> Showed high accuracy and was easy to interpret — a major advantage in clinical settings.</li>
  <li><b>SVM (RBF Kernel):</b> Handled nonlinear patterns well but lacked transparency in how predictions were made.</li>
</ul>

<p style="font-size: 0.6em; text-align: left; color: #444;">
From this, we learned that while all models had strengths, <b>Decision Tree</b> offered the best balance of performance and explainability for early CKD detection.
</p>


## <span style="font-size: 0.8em;">Model Comparison: Accuracy & Insights</span>
<br>
<pre style="font-size: 0.55em; text-align: left; background-color: #e6f4ea; padding: 8px 12px; border-radius: 5px; border: 1px solid #ccc; max-width: 600px; margin: 10px auto;">
1 dtc = DecisionTreeClassifier(random_state=42)
2 dtc.fit(X_train, y_train)
</pre>

<table style="font-size: 0.6em; margin: auto; border-collapse: collapse; width: 95%;">
  <thead>
    <tr style="background-color: #f0f0f0;">
      <th style="padding: 8px; border: 1px solid #ccc;">Model</th>
      <th style="padding: 8px; border: 1px solid #ccc;">Accuracy</th>
      <th style="padding: 8px; border: 1px solid #ccc;">Precision</th>
      <th style="padding: 8px; border: 1px solid #ccc;">Recall</th>
      <th style="padding: 8px; border: 1px solid #ccc;">F1-Score</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="padding: 8px; border: 1px solid #ccc;">SVM (RBF Kernel)</td>
      <td style="padding: 8px; border: 1px solid #ccc;">91%</td>
      <td style="padding: 8px; border: 1px solid #ccc;">0.90</td>
      <td style="padding: 8px; border: 1px solid #ccc;">0.91</td>
      <td style="padding: 8px; border: 1px solid #ccc;">0.90</td>
    </tr>
    <tr>
      <td style="padding: 8px; border: 1px solid #ccc;">K-Nearest Neighbors</td>
      <td style="padding: 8px; border: 1px solid #ccc;">95%</td>
      <td style="padding: 8px; border: 1px solid #ccc;">0.94</td>
      <td style="padding: 8px; border: 1px solid #ccc;">0.95</td>
      <td style="padding: 8px; border: 1px solid #ccc;">0.95</td>
    </tr>
    <tr style="background-color:rgb(216, 238, 218);">
      <td style="padding: 8px; border: 1px solid #ccc;"><b>Decision Tree </b></td>
      <td style="padding: 8px; border: 1px solid #ccc;"><b>99%</b></td>
      <td style="padding: 8px; border: 1px solid #ccc;"><b>0.99</b></td>
      <td style="padding: 8px; border: 1px solid #ccc;"><b>0.98</b></td>
      <td style="padding: 8px; border: 1px solid #ccc;"><b>0.99</b></td>
    </tr>
  </tbody>
</table>
<br>
<div style="background-color: #e6f4ea; border-left: 5px solid #1b5e20; padding: 12px 16px; border-radius: 6px; width: fit-content; margin: auto;">
  <p style="font-size: 0.55em; text-align: center; color: #444; margin: 0;">
    Based on accuracy and interpretability, <b>Decision Tree</b> was chosen as our final model for early CKD detection.
  </p>
</div>

---

## <span style="font-size: 0.6em;">Model Comparison: Accuracy, F1-Score & Interpretability</span>

:::{.columns} 

::: {.column width="50%"}


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define model metrics
labels = ['Accuracy', 'F1 Score', 'Interpretability']
num_vars = len(labels)

# Normalized scores (0–10 scale)
knn = [9.5, 9.5, 8]
dt = [10, 10, 10]
svm = [9.1, 9.0, 4]

# Prepare data
data = [knn, dt, svm]
model_names = ['KNN', 'Decision Tree', 'SVM (RBF)']
colors = ['#1f77b4', '#2ca02c', '#d62728']
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]

# Close the polygon
for i in range(len(data)):
    data[i] += data[i][:1]

# Radar chart
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

for i, d in enumerate(data):
    ax.plot(angles, d, color=colors[i], linewidth=2, label=model_names[i])
    ax.fill(angles, d, color=colors[i], alpha=0.25)

ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)
ax.set_thetagrids(np.degrees(angles[:-1]), labels)
ax.set_ylim(0, 10)
ax.set_title("Model Comparison Radar Chart", fontsize=16, weight='bold', y=1.08)
ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.1))

plt.tight_layout()
plt.show()

:::

::: {.column width="50%"}

<p style="font-size: 70%; color: #444;"> <strong>Decision Tree</strong> leads with perfect accuracy and interpretability, making it ideal for clinical deployment. <br>
<strong>KNN</strong> performs well and is simple, but less interpretable than DT in complex cases. <br>
<strong>SVM</strong> offers good predictive power, but its lack of transparency limits its usefulness in healthcare.<br>
Overall, Decision Tree balances performance and explainability — perfect for medical use-cases.
</p>

:::

:::

---

## <span style="font-size: 0.8em;">Confusion Matrix: KNN vs Decision Tree vs SVM</span>

<br>


In [ ]:
#| label: confusion-matrix-plot
#| fig-align: center

# Train/test split & feature setup
X = disease[['red_blood_cell_count', 'packed_cell_volume', 'haemoglobin', 'albumin', 'specific_gravity']]
y = disease['class']
X = X.fillna(X.mean())
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Import classifiers and libraries
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import seaborn as sns
import matplotlib.pyplot as plt

# Train models
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

svm_rbf = SVC(kernel='rbf', random_state=42)
svm_rbf.fit(X_train, y_train)

# Predict
knn_preds = knn.predict(X_test)
dtc_preds = dtc.predict(X_test)
svm_preds = svm_rbf.predict(X_test)

# Confusion matrices
knn_cm = confusion_matrix(y_test, knn_preds)
dtc_cm = confusion_matrix(y_test, dtc_preds)
svm_cm = confusion_matrix(y_test, svm_preds)

# Plotting
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
titles = ['KNN Confusion Matrix', 'Decision Tree Confusion Matrix', 'SVM (RBF) Confusion Matrix']
cms = [knn_cm, dtc_cm, svm_cm]
cmaps = ['YlGnBu', 'YlOrRd', 'PuBuGn']

for ax, cm, title, cmap in zip(axes, cms, titles, cmaps):
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, cbar=False,
                xticklabels=['CKD', 'Non-CKD'], yticklabels=['CKD', 'Non-CKD'],
                annot_kws={"size": 14}, ax=ax)
    ax.set_title(title, fontsize=14, weight='bold')
    ax.set_xlabel('Predicted Label', fontsize=12)
    ax.set_ylabel('Actual Label', fontsize=12)

plt.tight_layout()
plt.savefig("confusion_matrices_all.png", dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

<br>

<p style="font-size: 0.55em; text-align: center; color: #444;">
The Decision Tree model shows fewer misclassifications and better separation between CKD and Non-CKD cases.
</p>

---

## Thank You!  {.center}

<p style="text-align: center; font-size: 1.3em; color: #2c3e50; font-weight: bold;">
Early detection isn't just a technical solution — it’s a human one.  
</p>

<p style="text-align: center; font-size: 1em; color: #444;">
Through this project, we explored how <strong>data-driven insights</strong> and <strong>machine learning</strong>  
can support timely diagnosis and improve patient outcomes in Chronic Kidney Disease.
</p>

<hr style="width: 60%; margin: 20px auto; border: 0.5px solid #ccc;">

<p style="text-align: center; font-size: 0.95em; color: #555;">
Have questions or feedback? We'd love to hear from you!  
</p>

<p style="text-align: center; font-size: 0.85em; color: #888;">
Presented by: <strong>Shreemithra Naveen</strong> & <strong>Sai Laasya Gorantla</strong>  
<br>
INFO 511 – University of Arizona | April 2025
</p>
